In [1]:
import numpy as np
import polars as pl
import tensorflow as tf

from tensorflow.keras import layers

2023-05-10 01:14:59.441618: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-10 01:14:59.654614: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-10 01:14:59.659812: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-10 01:15:01.889329: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
from typing import Tuple

In [2]:
# tf.keras.utils.get_file('petfinder_mini.zip', dataset_url,
#                        extract=True, cache_dir='.')
csv_file = 'eda_ds.csv'
df = pl.read_csv(csv_file)
df.head()

tu_fico_9,tradeline_count,inquiry_count,collection_count,revolving_utilization,delinquent_secured,delinquent_unsecured,bankruptcy_count,stated_indiv_income,stated_additional_income,tu_income_estimate,tu_monthly_payments,tu_mortgage_payments,riskview,principal,cpb,days_delinquent,payment_history
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,str
498,1,0,1,0,0,0,0,24000,0,35,46,0,null,null,null,null,"""open;2019-05-1…"
612,7,5,1,63,0,0,0,30000,0,29,126,0,706,null,null,null,"""installment;20…"
617,21,8,0,32,0,0,0,60000,0,39,1186,1075,779,null,null,null,"""installment;20…"
647,10,1,0,64,0,0,0,400000,0,68,649,0,724,null,null,null,"""installment;20…"
647,16,7,4,56,0,0,0,65000,0,55,1074,0,660,null,null,null,"""installment;20…"


In [ ]:
# New column not based on existing
# df.with_columns(new_column = pl.lit('some_text'))

In [9]:
# In the original dataset, `'AdoptionSpeed'` of `4` indicates
# a pet was not adopted.
# df['target'] = np.where(dataframe['AdoptionSpeed']==4, 0, 1)
df = df.with_columns([(pl.col("stated_indiv_income")).alias("target")])
df.head()

# Drop unused features.
# df = df.drop(columns=['AdoptionSpeed', 'Description'])

tu_fico_9,tradeline_count,inquiry_count,collection_count,revolving_utilization,delinquent_secured,delinquent_unsecured,bankruptcy_count,stated_indiv_income,stated_additional_income,tu_income_estimate,tu_monthly_payments,tu_mortgage_payments,riskview,principal,cpb,days_delinquent,payment_history,target
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,str,i64
498,1,0,1,0,0,0,0,24000,0,35,46,0,null,null,null,null,"""open;2019-05-1…",24000
612,7,5,1,63,0,0,0,30000,0,29,126,0,706,null,null,null,"""installment;20…",30000
617,21,8,0,32,0,0,0,60000,0,39,1186,1075,779,null,null,null,"""installment;20…",60000
647,10,1,0,64,0,0,0,400000,0,68,649,0,724,null,null,null,"""installment;20…",400000
647,16,7,4,56,0,0,0,65000,0,55,1074,0,660,null,null,null,"""installment;20…",65000


In [69]:
df.select('target')

target
i64
24000
30000
60000
400000
65000
70000
27000
70000
81000


## Split the DataFrame into training, validation, and test sets

In [63]:
# train, val, test = np.split(df.sample(fraction=1), [int(0.8*len(df)), int(0.9*len(df))])

In [10]:
def train_test_split(
    df: pl.DataFrame, train_fraction: float = 0.75
) -> Tuple[pl.DataFrame, pl.DataFrame, pl.DataFrame]:
    """Split polars dataframe into two sets.
    Args:
        df (pl.DataFrame): Dataframe to split
        train_fraction (float, optional): Fraction that goes to train. Defaults to 0.75.
    Returns:
        Tuple[pl.DataFrame, pl.DataFrame]: Tuple of train and test dataframes
    """
    df = df.with_columns(pl.all().shuffle(seed=1))
    split_index = int(train_fraction * len(df))
    df_train = df[:split_index]
    df_test = df[split_index:]
    half = int(len(df_test)/2)
    val = df_test[half:]
    test = df_test[:half]
    return (df_train, test, val)

In [11]:
train, test, val = train_test_split(df)

In [12]:
print(len(train), 'training examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

74 training examples
13 validation examples
12 test examples


In [70]:
train = pl.DataFrame(train)

## Create input pipeline using tf.data
Create a utility function that converts each training, validation, and test set DataFrame into a `tf.data.Dataset`, then shuffles and batches the data.
If you were working with a very large CSV file (so large that it does not fit into memory), you'd use the `tf.data` API to read it from disk directly.

In [13]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  df = dataframe
  labels = df.select('target')
  df = {key: value[:,tf.newaxis] for key, value in dataframe.items()}
  ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds

Now, use newly created function (`df_to_dataset`) to check format of data that input pipeline helper function returns by calling it on the training data, & use a small batch size to keep the output readable:

In [14]:
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)

AttributeError: 'DataFrame' object has no attribute 'items'

In [ ]:
[(train_features, label_batch)] = train_ds.take(1)
print('Every feature:', list(train_features.keys()))
print('A batch of ages:', train_features['Age'])
print('A batch of targets:', label_batch )